In [526]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import numpy as np
import torch.nn as nn
import torch
from causalml.inference.meta import XGBTRegressor

### Loading data

In [9]:
#clients id, treatment and their target
Train_clients = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/train.csv')
print(Train_clients.info())
Train_clients.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140027 entries, 0 to 140026
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   client_id      140027 non-null  object
 1   treatment_flg  140027 non-null  int64 
 2   target         140027 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.2+ MB
None


,client_id,treatment_flg,target
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1
3,e99e6fabb9,0,0
4,27fb6f8520,1,1


In [50]:
Test_clients = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/test.csv')
print(Test_clients.info())
Test_clients.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60012 entries, 0 to 60011
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  60012 non-null  object
dtypes: object(1)
memory usage: 469.0+ KB
None


,client_id
0,a9a604ed6e
1,ebd7360016
2,908cd9b8e8
3,dceb8ce861
4,f4f0ac6b06


In [ ]:
all_clients_info = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/clients2.csv', usecols = ['client_id','first_issue_date','first_redeem_date','age','gender'])
print(all_clients_info.info())
all_clients_info.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200039 entries, 0 to 200038
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   client_id          200039 non-null  object
 1   first_issue_date   200039 non-null  object
 2   first_redeem_date  182493 non-null  object
 3   age                200039 non-null  int64 
 4   gender             200039 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.6+ MB
None


,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [11]:
all_products = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/products.csv')
print(all_products.info())
all_products.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43038 entries, 0 to 43037
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        43038 non-null  object 
 1   level_1           43035 non-null  object 
 2   level_2           43035 non-null  object 
 3   level_3           43035 non-null  object 
 4   level_4           43035 non-null  object 
 5   segment_id        41466 non-null  float64
 6   brand_id          37838 non-null  object 
 7   vendor_id         43004 non-null  object 
 8   netto             43035 non-null  float64
 9   is_own_trademark  43038 non-null  int64  
 10  is_alcohol        43038 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 3.6+ MB
None


,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


In [12]:
train_purchases = pd.read_csv('/home/luchian/all_data/uni_data/upliftX5_data/train_purch.csv')

In [13]:
print(train_purchases.info())
train_purchases.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15998952 entries, 0 to 15998951
Data columns (total 13 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   client_id                object 
 1   transaction_id           object 
 2   transaction_datetime     object 
 3   regular_points_received  float64
 4   express_points_received  float64
 5   regular_points_spent     float64
 6   express_points_spent     float64
 7   purchase_sum             float64
 8   store_id                 object 
 9   product_id               object 
 10  product_quantity         float64
 11  trn_sum_from_iss         float64
 12  trn_sum_from_red         float64
dtypes: float64(8), object(5)
memory usage: 1.5+ GB
None


,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


### Data Processing

In [ ]:
#it took about 6 minutes execute (need to save this)
# train_clients_mask = all_clients_info['client_id'].map(lambda x: Train_clients['client_id'].isin([x]).any())
# train_clients_mask

0          True
1          True
2         False
3          True
4          True
          ...  
200034     True
200035    False
200036    False
200037     True
200038    False
Name: client_id, Length: 200039, dtype: bool

In [ ]:
#saving it ... 
# train_clients_info = all_clients_info[train_clients_mask]
# train_clients_info.head(5)

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U
5,000220a0a7,2017-12-09 17:26:20,2018-10-04 14:59:29,115,M


In [612]:
#loading info of training calients
train_clients_info = pd.read_csv('train_clients_info.csv')
train_clients_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140027 entries, 0 to 140026
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   client_id          140027 non-null  object
 1   first_issue_date   140027 non-null  object
 2   first_redeem_date  127673 non-null  object
 3   age                140027 non-null  int64 
 4   gender             140027 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.3+ MB


In [613]:
class DateTimeSplitter(BaseEstimator, TransformerMixin):
    """
    splits one of the datetime dtype column into separate columns
    """
    
    def __init__(self,
                 input_year = 1991,
                 components=['year','month','day','hour','minute','second'],
                 errors='coerce',
                 utc=False):
        self.input_year = input_year
        self.components = components
        self.errors = errors
        self.utc = utc
        
    def fit(self, X, y=None):
        """Store input feature names"""
        if not isinstance(X, np.ndarray):
            raise ValueError("Input must be numpy array")
            
        # Store valid components
        valid_components = ['year','month','day','hour','minute','second','dayofweek','quarter']
        self.components_ = [c for c in self.components if c in valid_components]
        return self
        
    def transform(self, X):
        """Split datetime column into components"""
        check_is_fitted(self)
        X = X.copy()
        # Convert to datetime
        datetime_series = pd.to_datetime(pd.Series(X.reshape(-1)))
        
        # Extract components
        component_data = {}
        for comp in self.components_:
            if comp == 'year':
                component_data[comp] = datetime_series.dt.year
            elif comp == 'month':
                component_data[comp] = datetime_series.dt.month
            elif comp == 'day':
                component_data[comp] = datetime_series.dt.day
            elif comp == 'hour':
                component_data[comp] = datetime_series.dt.hour
            elif comp == 'minute':
                component_data[comp] = datetime_series.dt.minute
            elif comp == 'second':
                component_data[comp] = datetime_series.dt.second
            elif comp == 'dayofweek':
                component_data[comp] = datetime_series.dt.dayofweek
            elif comp == 'quarter':
                component_data[comp] = datetime_series.dt.quarter
        
        ret_value = pd.DataFrame(component_data).to_numpy()
        ret_value[ret_value[:,0] == self.input_year] = np.zeros(len(self.components_),dtype = np.int8)
        return ret_value
    
    def get_feature_names_out(self):
        return self.components_

In [621]:
def process_dates(the_frame,cols):
    """Applies simple transformaions to datetime columns"""
    input_date = '1921-11-11 20:20:20'
    result_frame = the_frame.copy()
    list_of_frames = []
    #the main pipeline
    for col in cols:
        fill_nan = ColumnTransformer([
            ('with_dummy', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = input_date), [col]),
        ])
        date_trans = Pipeline([
            ('initial_trans',fill_nan),
            ('splitted_date',DateTimeSplitter(input_year=1921))
        ])
        #for each column apply the transformation
        #and concatenate afterwards
        result = date_trans.fit_transform(the_frame)
        col_names = [col+'_'+ date_part for date_part in date_trans[-1].get_feature_names_out()]
        list_of_frames.append(pd.DataFrame(result,columns=col_names))
        result_frame.drop(col,inplace = True,axis = 1)
    #concatenate the cols and drop the old ones
    for one_frame in list_of_frames:
        result_frame = pd.concat([result_frame,one_frame],axis = 1)
    return result_frame


In [628]:
process_dates(all_clients_info,cols = ['first_issue_date','first_redeem_date'])

,client_id,age,gender,first_issue_date_year,first_issue_date_month,first_issue_date_day,first_issue_date_hour,first_issue_date_minute,first_issue_date_second,first_redeem_date_year,first_redeem_date_month,first_redeem_date_day,first_redeem_date_hour,first_redeem_date_minute,first_redeem_date_second
0,000012768d,45,U,2017,8,5,15,40,48,2018,1,4,19,30,7
1,000036f903,72,F,2017,4,10,13,54,23,2017,4,23,12,37,56
2,00010925a5,83,U,2018,7,24,16,21,29,2018,9,14,16,12,49
3,0001f552b0,33,F,2017,6,30,19,20,38,2018,8,28,12,59,45
4,00020e7b18,73,U,2017,11,27,11,41,45,2018,1,10,17,50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200034,fffe0abb97,35,F,2017,11,27,8,56,54,2018,2,11,9,26,8
200035,fffe0ed719,69,U,2017,9,15,8,53,24,2017,12,12,14,50,12
200036,fffea1204c,73,F,2018,1,31,16,59,37,2018,3,12,17,2,27
200037,fffeca6d22,77,F,2017,12,28,11,56,13,0,0,0,0,0,0


In [627]:
some_ar = np.array([k for k in range(1,101)])
print(some_ar)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


### baseline models

In [15]:
from causalml.inference.meta import XGBTRegressor

In [16]:
type(XGBTRegressor)

abc.ABCMeta